## Configuration

In [ ]:
import subprocess
import os
print('Current conda environment:', os.environ['CONDA_DEFAULT_ENV'])
os.environ['TOKENIZERS_PARALLELISM'] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # set gpu

os.environ["PATH"] += ":/usr/local/openeye/bin"
os.environ["OE_LICENSE"] = "/home/fts_g_ucla_edu/Projects/oe_license.txt"

cwd = os.getcwd()
print('Working directory:', cwd)

import warnings
warnings.filterwarnings('ignore')

import random
random.seed(42)

In [ ]:
import numpy as np

import pandas as pd

from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import MolsToGridImage
import useful_rdkit_utils as uru
from rdkit.Chem.rdFMCS import FindMCS

import pickle

import mols2grid

## Generating Data

In [ ]:
models = ['reinvent', 'coati', 'safe']
prefixes = ['2zdt', '2qd9', '2ojg']

In [ ]:
%%capture

reinvent_distributions, crem_distributions, coati_distributions, safe_distributions = [], [], [], []

for pdb in prefixes:
    for model in models:

        DF_FILEPATH = f'data/{model}_dataframe.csv'

        arg1 = '--model'
        arg2 = '--sample'
        arg3 = '--remove_odd_rings'
        arg4 = '--dock'
        arg5 = '--pdb'

        args = ['python3', 'generate_analogs.py',
                arg1, model,
                arg2, '200',
                arg3,
                arg4,
                arg5, pdb]

        # Change directory to generate analogs with python script
        %cd ..

        subprocess.run(args,
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.STDOUT)
                
        # Change directory back to that of the current notebook
        %cd experiments

        df = pd.read_csv(DF_FILEPATH, index_col=0)

        df['Model'] = model

        if model == 'reinvent':
            reinvent_distributions.append(df)
        elif model == 'crem':
            crem_distributions.append(df)
        elif model == 'coati':
            coati_distributions.append(df)
        elif model == 'safe':
            safe_distributions.append(df)

In [ ]:
data = {'reinvent' : reinvent_distributions,
        'crem' : crem_distributions,
        'coati' : coati_distributions,
        'safe' : safe_distributions}

In [ ]:
with open('lists_docking.pkl', 'wb') as file:
    pickle.dump(data, file)

## Docking